# 🫀 CNN1D + Transformer para Clasificación Binaria Supervisada de ECG

Este notebook implementa un **CNN1D + Transformer Encoder** para clasificación binaria supervisada (ECG normal vs anómalo).

**Características principales:**
- Arquitectura híbrida: CNN1D para extracción de características locales + Transformer Encoder para dependencias temporales con atención
- Entrenamiento supervisado con etiquetas (0=normal, 1=anómalo)
- Datos preprocesados desde `Datos_supervisados/tensors_200hz` (archivos .pt)
- Integración con MLflow para tracking de experimentos
- Orquestación con Prefect 2.x
- Soporte automático para GPU (RTX 5080 compatible)

> ⚠️ **IMPORTANTE EN WINDOWS:** Ejecuta la celda de **Setup DLLs CUDA** (celda 2) **ANTES** de la celda de imports. Esto es necesario para que PyTorch pueda cargar las DLLs de CUDA correctamente.

> ▶️ **Instrucciones:** 
> 1. Ejecuta la celda de **Setup DLLs CUDA** primero
> 2. Configura los parámetros en la sección de **CONFIGURACIÓN GENERAL**
> 3. Ajusta la ruta `DATA_DIR` a tu carpeta de datos (debe apuntar a `Datos_supervisados/tensors_200hz`)
> 4. Ejecuta todas las demás celdas en orden


## 📋 Índice

1. **Setup CUDA y dependencias** - Configuración de DLLs y librerías
2. **Configuración general** - Imports, semillas, dispositivo, hiperparámetros
3. **Carga y preparación de datos** - Funciones para cargar desde `tensors_200hz`
4. **Definición del modelo CNN1D + Transformer** - Arquitectura híbrida
5. **Funciones de entrenamiento y evaluación** - Loops de entrenamiento y validación
6. **Integración con MLflow** - Configuración y logging
7. **Orquestación con Prefect** - Flujo principal con Prefect
8. **Ejecución del flujo completo** - Celda final para ejecutar todo
9. **Guardado de modelo** - Exportar modelo estándar y para AWS SageMaker


---

## 1. ⚙️ Setup CUDA y Dependencias


In [63]:
# ========================================
# 🔧 Setup RTX 5080 — dependencias + CUDA DLL
# Ejecuta una sola vez (o tras actualizar drivers/librerías)
# ========================================
import os
import sys
import subprocess
from pathlib import Path
from textwrap import dedent

print(f"Python: {sys.executable}")
print(f"Working dir: {Path.cwd().resolve()}")

# Rutas candidatas para DLLs de CUDA
CUDA_CANDIDATES = [
    os.environ.get("CUDA_PATH"),
    os.environ.get("CUDA_PATH_V12_8"),
    r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8",
    r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\bin",
    r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\libnvvp",
    r"C:\Program Files\NVIDIA\CUDNN",
]

# Añadir rutas DLL en Windows (necesario antes de importar torch)
added = []
if hasattr(os, "add_dll_directory"):
    for candidate in CUDA_CANDIDATES:
        if not candidate:
            continue
        path = Path(candidate)
        if path.is_dir():
            try:
                os.add_dll_directory(str(path))
                added.append(str(path))
            except (FileNotFoundError, OSError):
                pass

if added:
    print("DLL directories añadidos:")
    for path in added:
        print(f"  - {path}")

# Instalar dependencias base si no están instaladas
BASE_PACKAGES = [
    "mlflow>=2.16",
    "prefect>=3",
    "scikit-learn",
    "matplotlib",
    "pandas",
    "numpy",
    "seaborn",
]

def pip_install(spec: str) -> None:
    module_name = spec.split("==")[0].split("[")[0].replace("-", "_")
    try:
        __import__(module_name)
        print(f"✔ {spec} ya instalado")
    except Exception:
        print(f"⏳ Instalando {spec} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", spec])

for pkg in BASE_PACKAGES:
    pip_install(pkg)

# Comando para instalar PyTorch nightly con CUDA 12.8 (para RTX 5080)
TORCH_INSTALL_CMD = [
    sys.executable,
    "-m",
    "pip",
    "install",
    "--upgrade",
    "--pre",
    "torch",
    "torchvision",
    "torchaudio",
    "--index-url",
    "https://download.pytorch.org/whl/nightly/cu128",
]

def ensure_torch_cuda() -> "tuple[object | None, dict]":
    """Importa torch, o instala la nightly cu128 si hace falta."""
    info: dict[str, str | float | bool] = {}
    try:
        import torch  # type: ignore
        info["torch_version"] = getattr(torch, "__version__", "desconocida")
        info["cuda_version"] = getattr(getattr(torch, "version", object()), "cuda", "desconocida")
        info["cuda_available"] = bool(torch.cuda.is_available())
        if "cu128" not in info["torch_version"] and not str(info["cuda_version"]).startswith("12.8"):
            raise RuntimeError(
                f"Build {info['torch_version']} no es cu128. Se reinstalará la nightly para RTX 5080."
            )
        return torch, info
    except Exception as err:
        print("⚠️ Torch no usable todavía:", err)
        print("   Desinstalando PyTorch corrupto...")
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "torch", "torchvision", "torchaudio"])
        print("   Instalando nightly cu128 desde PyTorch (puede tardar).")
        subprocess.check_call(TORCH_INSTALL_CMD)
        print("\n" + "="*60)
        print("⚠️ IMPORTANTE: PyTorch fue reinstalado.")
        print("   DEBES REINICIAR EL KERNEL DE JUPYTER ahora:")
        print("   Kernel → Restart Kernel")
        print("   Luego ejecuta esta celda de nuevo.")
        print("="*60)
        import importlib
        import time
        time.sleep(2)
        importlib.invalidate_caches()
        try:
            import torch  # type: ignore
            info["torch_version"] = getattr(torch, "__version__", "desconocida")
            info["cuda_version"] = getattr(getattr(torch, "version", object()), "cuda", "desconocida")
            info["cuda_available"] = bool(torch.cuda.is_available())
            return torch, info
        except Exception as e2:
            print(f"\n❌ No se pudo importar PyTorch después de reinstalar: {e2}")
            print("   Por favor, REINICIA EL KERNEL y ejecuta esta celda de nuevo.")
            raise RuntimeError("Reinicia el kernel de Jupyter y ejecuta esta celda de nuevo.") from e2

# Intentar importar/instalar PyTorch
torch, torch_info = ensure_torch_cuda()

print("\nTorch info:")
for k, v in torch_info.items():
    print(f"  - {k}: {v}")

if torch_info.get("cuda_available"):
    try:
        gpu_name = torch.cuda.get_device_name(0)
        cc = torch.cuda.get_device_properties(0)
        print(f"GPU detectada: {gpu_name} | SM {cc.major}{cc.minor}")
    except Exception as e:
        print("⚠️ CUDA disponible pero no se pudo consultar GPU:", e)
else:
    print(dedent(
        """
        ⚠️ CUDA sigue inactiva. Revisa drivers / reinicia kernel tras la instalación.
        Si el problema continúa, ejecuta manualmente:
          pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128
        """
    ))


Python: c:\Python311\python.exe
Working dir: S:\Proyecto final\Books
DLL directories añadidos:
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\bin
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\libnvvp
⏳ Instalando mlflow>=2.16 ...
⏳ Instalando prefect>=3 ...
⏳ Instalando scikit-learn ...
✔ matplotlib ya instalado
✔ pandas ya instalado
✔ numpy ya instalado
✔ seaborn ya instalado

Torch info:
  - torch_version: 2.10.0.dev20251121+cu128
  - cuda_version: 12.8
  - cuda_available: True
GPU detectada: NVIDIA GeForce RTX 5080 | SM 120


In [64]:
# ========================================
# Imports y dependencias
# ========================================
# ⚠️ IMPORTANTE: Ejecuta la celda anterior (Setup DLLs) antes de esta celda
# torch ya está importado en la celda anterior
import random
import json
import time
import math
from pathlib import Path
from typing import Tuple, Dict, List, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# torch ya está importado en la celda anterior, solo importamos los submódulos
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    accuracy_score,
    classification_report,
)
import mlflow
import mlflow.pytorch
from prefect import task, flow
from prefect.tasks import NO_CACHE

# Instalar seaborn si no está disponible (para las visualizaciones de validación)
try:
    import seaborn as sns
except ImportError:
    print("⏳ Instalando seaborn...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "seaborn"])
    import seaborn as sns
    print("✓ seaborn instalado")

print("✓ Todos los imports completados")


✓ Todos los imports completados


---

## 2. ⚙️ Configuración General


In [65]:
# ========================================
# CONFIGURACIÓN GENERAL
# ========================================

# --- Rutas y nombres ---
DATA_DIR = Path("../data/Datos_supervisados/tensors_200hz")  # TODO: Ajustar ruta si es necesario
EXPERIMENT_NAME = "ecg_cnn_transformer_supervisado"    # Nombre del experimento en MLflow
RUN_NAME = "cnn_transformer_ecg_v1"          # Nombre del run en MLflow
OUTPUT_DIR = Path("./outputs")                 # Directorio para guardar artefactos

# --- Datos de entrada ---
N_CHANNELS = 3          # derivaciones de ECG (3 canales)
SEQ_LEN = 2000          # TODO: timesteps por ejemplo (ej: 10 s a 200 Hz => 2000)

# --- Arquitectura CNN 1D ---
# Cada entrada en estas listas define una capa Conv1d:
# - out_channels_list[i]: filtros de la capa i
# - kernel_sizes[i]: tamaño de kernel para la capa i
# - pool_sizes[i]: tamaño del MaxPool1d posterior (si se usa)
out_channels_list = [32, 64, 128]   # TODO: ajustar cantidad de filtros por capa
kernel_sizes = [7, 5, 3]           # TODO: ajustar tamaños de kernel
pool_sizes = [2, 2, 2]             # TODO: ajustar factor de pooling por capa (o None si no se usa)
USE_BATCHNORM = True                # aplicar BatchNorm1d después de cada conv
CNN_ACTIVATION = "relu"             # usar ReLU

# --- Arquitectura TRANSFORMER ---
# d_model será la dimensión de características por timestep luego del bloque CNN
D_MODEL = 256                        # dimensión del espacio de representación para el Transformer
NHEAD = 4                           # número de cabezas de atención
NUM_ENCODER_LAYERS = 3              # cantidad de capas TransformerEncoder
DIM_FEEDFORWARD = 512               # dimensión de la capa feedforward interna
TRANSFORMER_DROPOUT = 0.1           # dropout dentro del Transformer
USE_POS_ENCODING = True             # aplicar codificación posicional senoidal

# --- Capa totalmente conectada ---
FC_UNITS = 128                       # tamaño de la capa lineal antes de la salida

# --- Entrenamiento ---
BATCH_SIZE = 128
LEARNING_RATE = 3e-4
NUM_EPOCHS = 50
WEIGHT_DECAY = 1e-5                 # regularización L2 (0.0 si no se quiere)
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_STEPS = 500                 # pasos de warmup para el learning rate
USE_SCHEDULER = True
SCHEDULER_PATIENCE = 5
SCHEDULER_FACTOR = 0.5
SCHEDULER_MIN_LR = 1e-6
SCHEDULER_MODE = 'max'  # Monitorear val_f1_macro (maximizar)
CLIP_GRAD_NORM = 1.0

# --- Otros ---
SEED = 42
USE_CUDA = True                     # si hay GPU disponible, usarla
ENABLE_CUDNN_BENCHMARK = True

# --- MLflow ---
MLFLOW_TRACKING_URI = None  # None = usa el directorio local (sqlite:///mlflow.db)

# Crear directorio de salida
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Diccionario de configuración (para pasar a funciones)
CONFIG = {
    "DATA_DIR": DATA_DIR,
    "EXPERIMENT_NAME": EXPERIMENT_NAME,
    "RUN_NAME": RUN_NAME,
    "OUTPUT_DIR": OUTPUT_DIR,
    "N_CHANNELS": N_CHANNELS,
    "SEQ_LEN": SEQ_LEN,
    "out_channels_list": out_channels_list,
    "kernel_sizes": kernel_sizes,
    "pool_sizes": pool_sizes,
    "USE_BATCHNORM": USE_BATCHNORM,
    "CNN_ACTIVATION": CNN_ACTIVATION,
    "D_MODEL": D_MODEL,
    "NHEAD": NHEAD,
    "NUM_ENCODER_LAYERS": NUM_ENCODER_LAYERS,
    "DIM_FEEDFORWARD": DIM_FEEDFORWARD,
    "TRANSFORMER_DROPOUT": TRANSFORMER_DROPOUT,
    "USE_POS_ENCODING": USE_POS_ENCODING,
    "FC_UNITS": FC_UNITS,
    "BATCH_SIZE": BATCH_SIZE,
    "LEARNING_RATE": LEARNING_RATE,
    "NUM_EPOCHS": NUM_EPOCHS,
    "WEIGHT_DECAY": WEIGHT_DECAY,
    "GRADIENT_ACCUMULATION_STEPS": GRADIENT_ACCUMULATION_STEPS,
    "USE_SCHEDULER": USE_SCHEDULER,
    "WARMUP_STEPS": WARMUP_STEPS,
    "SCHEDULER_PATIENCE": SCHEDULER_PATIENCE,
    "SCHEDULER_FACTOR": SCHEDULER_FACTOR,
    "SCHEDULER_MIN_LR": SCHEDULER_MIN_LR,
    "SCHEDULER_MODE": SCHEDULER_MODE,
    "CLIP_GRAD_NORM": CLIP_GRAD_NORM,
    "SEED": SEED,
    "USE_CUDA": USE_CUDA,
    "ENABLE_CUDNN_BENCHMARK": ENABLE_CUDNN_BENCHMARK,
    "MLFLOW_TRACKING_URI": MLFLOW_TRACKING_URI,
}

print("✓ Configuración cargada:")
print(json.dumps({k: str(v) if isinstance(v, Path) else v for k, v in CONFIG.items()}, indent=2, ensure_ascii=False))


✓ Configuración cargada:
{
  "DATA_DIR": "..\\data\\Datos_supervisados\\tensors_200hz",
  "EXPERIMENT_NAME": "ecg_cnn_transformer_supervisado",
  "RUN_NAME": "cnn_transformer_ecg_v1",
  "OUTPUT_DIR": "outputs",
  "N_CHANNELS": 3,
  "SEQ_LEN": 2000,
  "out_channels_list": [
    32,
    64,
    128
  ],
  "kernel_sizes": [
    7,
    5,
    3
  ],
  "pool_sizes": [
    2,
    2,
    2
  ],
  "USE_BATCHNORM": true,
  "CNN_ACTIVATION": "relu",
  "D_MODEL": 256,
  "NHEAD": 4,
  "NUM_ENCODER_LAYERS": 3,
  "DIM_FEEDFORWARD": 512,
  "TRANSFORMER_DROPOUT": 0.1,
  "USE_POS_ENCODING": true,
  "FC_UNITS": 128,
  "BATCH_SIZE": 128,
  "LEARNING_RATE": 0.0003,
  "NUM_EPOCHS": 50,
  "WEIGHT_DECAY": 1e-05,
  "GRADIENT_ACCUMULATION_STEPS": 1,
  "USE_SCHEDULER": true,
  "WARMUP_STEPS": 500,
  "SCHEDULER_PATIENCE": 5,
  "SCHEDULER_FACTOR": 0.5,
  "SCHEDULER_MIN_LR": 1e-06,
  "SCHEDULER_MODE": "max",
  "CLIP_GRAD_NORM": 1.0,
  "SEED": 42,
  "USE_CUDA": true,
  "ENABLE_CUDNN_BENCHMARK": true,
  "MLFLOW_TRAC

In [66]:
# ========================================
# Configuración de semillas aleatorias y optimizaciones GPU
# ========================================
def set_seed_everywhere(seed: int = 42, enable_cudnn_benchmark: bool = True) -> None:
    """Fija semillas para reproducibilidad y optimiza GPU."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = enable_cudnn_benchmark
        torch.cuda.empty_cache()
        print(f"✓ cuDNN Benchmark: {'Habilitado' if enable_cudnn_benchmark else 'Deshabilitado'}")

set_seed_everywhere(SEED, enable_cudnn_benchmark=CONFIG.get("ENABLE_CUDNN_BENCHMARK", True))
print(f"✓ Semilla fijada: {SEED}")


✓ cuDNN Benchmark: Habilitado
✓ Semilla fijada: 42


In [67]:
# ========================================
# Configuración de dispositivo (GPU/CPU)
# ========================================
def get_device() -> torch.device:
    """Detecta y configura el dispositivo (GPU si está disponible)."""
    if USE_CUDA and torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✓ GPU detectada: {gpu_name}")
        print(f"  CUDA Version: {torch.version.cuda}")
        print(f"  PyTorch Version: {torch.__version__}")
    else:
        device = torch.device("cpu")
        print("⚠ GPU no disponible, usando CPU")
    return device

DEVICE = get_device()
print(f"Dispositivo seleccionado: {DEVICE}")


✓ GPU detectada: NVIDIA GeForce RTX 5080
  CUDA Version: 12.8
  PyTorch Version: 2.10.0.dev20251121+cu128
Dispositivo seleccionado: cuda


---

## 3. 📂 Carga y Preparación de Datos


In [68]:
# ========================================
# Función para cargar datos desde tensors_200hz (archivos .pt)
# ========================================
def load_tensor_data(
    data_dir: Path,
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Carga los datos desde archivos .pt en tensors_200hz.
    
    Args:
        data_dir: Ruta a la carpeta tensors_200hz
        
    Returns:
        Tuple con (X_train, y_train, X_val, y_val, X_test, y_test)
    """
    print("="*70)
    print("📂 CARGANDO DATOS DESDE tensors_200hz")
    print("="*70)
    print(f"Directorio: {data_dir.resolve()}")
    
    # Cargar tensors
    print("\n⏳ Cargando tensors desde disco...")
    X_train = torch.load(data_dir / "X_train.pt", map_location='cpu')
    y_train = torch.load(data_dir / "y_train.pt", map_location='cpu')
    X_val = torch.load(data_dir / "X_val.pt", map_location='cpu')
    y_val = torch.load(data_dir / "y_val.pt", map_location='cpu')
    X_test = torch.load(data_dir / "X_test.pt", map_location='cpu')
    y_test = torch.load(data_dir / "y_test.pt", map_location='cpu')
    
    # Verificar formas
    print(f"\n✓ Datos cargados:")
    print(f"  X_train: {X_train.shape} | y_train: {y_train.shape} (normales: {(y_train==0).sum().item()}, anómalos: {(y_train==1).sum().item()})")
    print(f"  X_val:   {X_val.shape} | y_val:   {y_val.shape} (normales: {(y_val==0).sum().item()}, anómalos: {(y_val==1).sum().item()})")
    print(f"  X_test:  {X_test.shape} | y_test:  {y_test.shape} (normales: {(y_test==0).sum().item()}, anómalos: {(y_test==1).sum().item()})")
    
    # Verificar que las features coincidan con CONFIG
    if X_train.shape[-1] != CONFIG["N_CHANNELS"]:
        print(f"⚠️ ADVERTENCIA: X_train tiene {X_train.shape[-1]} canales, pero N_CHANNELS={CONFIG['N_CHANNELS']}")
        print(f"   Ajustando N_CHANNELS a {X_train.shape[-1]}")
        CONFIG["N_CHANNELS"] = X_train.shape[-1]
    
    # Verificar SEQ_LEN
    if len(X_train.shape) >= 2 and X_train.shape[1] != CONFIG["SEQ_LEN"]:
        print(f"⚠️ ADVERTENCIA: X_train tiene {X_train.shape[1]} timesteps, pero SEQ_LEN={CONFIG['SEQ_LEN']}")
        print(f"   Ajustando SEQ_LEN a {X_train.shape[1]}")
        CONFIG["SEQ_LEN"] = X_train.shape[1]
    
    print("="*70)
    
    return X_train, y_train, X_val, y_val, X_test, y_test


def create_dataloaders_from_tensors(
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    X_test: torch.Tensor,
    y_test: torch.Tensor,
    batch_size: int,
    shuffle_train: bool = True,
) -> Tuple[DataLoader, DataLoader, DataLoader]:
    """
    Crea DataLoaders desde tensors.
    
    Returns:
        Tuple con (train_loader, val_loader, test_loader)
    """
    # Crear datasets
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)
    
    # Crear dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=shuffle_train,
        num_workers=0,  # 0 para Windows
        pin_memory=torch.cuda.is_available(),
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=torch.cuda.is_available(),
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=torch.cuda.is_available(),
    )
    
    print(f"\n✓ DataLoaders creados:")
    print(f"  Train: {len(train_loader)} batches ({len(train_dataset)} muestras)")
    print(f"  Val:   {len(val_loader)} batches ({len(val_dataset)} muestras)")
    print(f"  Test:  {len(test_loader)} batches ({len(test_dataset)} muestras)")
    
    return train_loader, val_loader, test_loader


In [69]:
# ========================================
# Clase para Codificación Posicional Senoidal
# ========================================
class PositionalEncoding(nn.Module):
    """
    Codificación posicional senoidal para Transformer.
    """
    def __init__(self, d_model: int, max_len: int = 5000, dropout: float = 0.1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Crear la matriz de codificación posicional
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)  # (max_len, 1, d_model)
        
        # Registrar como buffer (no es un parámetro entrenable)
        self.register_buffer('pe', pe)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor de forma (seq_len, batch, d_model)
        Returns:
            Tensor con codificación posicional añadida
        """
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


In [70]:
# ========================================
# Clase CNN1D + Transformer para Clasificación Binaria
# ========================================
class CNN1D_TransformerClassifier(nn.Module):
    """
    CNN1D + Transformer Encoder para clasificación binaria de series temporales (ECG normal vs anómalo).
    
    Arquitectura:
    - Bloque CNN1D: Extracción de características locales
    - Transformer Encoder: Captura de dependencias temporales con atención
    - Capa fully connected
    - Salida binaria (sigmoid)
    """
    
    def __init__(
        self,
        n_channels: int,
        seq_len: int,
        out_channels_list: List[int],
        kernel_sizes: List[int],
        pool_sizes: List[int],
        use_batchnorm: bool = True,
        cnn_activation: str = "relu",
        d_model: int = 64,
        nhead: int = 4,
        num_encoder_layers: int = 2,
        dim_feedforward: int = 128,
        transformer_dropout: float = 0.1,
        use_pos_encoding: bool = True,
        fc_units: int = 32,
    ):
        super(CNN1D_TransformerClassifier, self).__init__()
        
        self.n_channels = n_channels
        self.seq_len = seq_len
        self.out_channels_list = out_channels_list
        self.num_cnn_layers = len(out_channels_list)
        self.d_model = d_model
        self.use_pos_encoding = use_pos_encoding
        
        # ========================================
        # Bloque CNN 1D
        # ========================================
        cnn_layers = []
        in_channels = n_channels
        
        for i in range(self.num_cnn_layers):
            # Conv1d
            cnn_layers.append(
                nn.Conv1d(
                    in_channels=in_channels,
                    out_channels=out_channels_list[i],
                    kernel_size=kernel_sizes[i],
                    padding=(kernel_sizes[i] - 1) // 2,  # padding 'same'
                )
            )
            
            # BatchNorm (si está habilitado)
            if use_batchnorm:
                cnn_layers.append(nn.BatchNorm1d(out_channels_list[i]))
            
            # Activación
            if cnn_activation.lower() == "relu":
                cnn_layers.append(nn.ReLU())
            elif cnn_activation.lower() == "leakyrelu":
                cnn_layers.append(nn.LeakyReLU(0.1))
            else:
                raise ValueError(f"Activación {cnn_activation} no soportada")
            
            # MaxPool (si está definido)
            if pool_sizes[i] is not None and pool_sizes[i] > 1:
                cnn_layers.append(nn.MaxPool1d(kernel_size=pool_sizes[i]))
            
            in_channels = out_channels_list[i]
        
        self.cnn = nn.Sequential(*cnn_layers)
        
        # Calcular tamaño de salida de CNN después del pooling
        # Hacemos un forward dummy para determinar el tamaño de salida
        with torch.no_grad():
            dummy_input = torch.zeros(1, n_channels, seq_len)
            cnn_output = self.cnn(dummy_input)
            # cnn_output shape: (batch, out_channels, seq_len_reduced)
            self.cnn_output_seq_len = cnn_output.shape[2]
            self.cnn_output_channels = cnn_output.shape[1]
        
        # Proyección lineal para ajustar la dimensión a d_model
        # Si cnn_output_channels != d_model, necesitamos proyectar
        if self.cnn_output_channels != d_model:
            self.cnn_projection = nn.Linear(self.cnn_output_channels, d_model)
        else:
            self.cnn_projection = nn.Identity()
        
        # ========================================
        # Transformer Encoder
        # ========================================
        # La entrada al Transformer es (seq_len, batch, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=transformer_dropout,
            activation='relu',
            batch_first=False,  # (seq_len, batch, features) para positional encoding
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_encoder_layers,
        )
        
        # Codificación posicional
        if use_pos_encoding:
            self.pos_encoder = PositionalEncoding(d_model, max_len=seq_len, dropout=transformer_dropout)
        else:
            self.pos_encoder = None
        
        # ========================================
        # Capas Fully Connected
        # ========================================
        # Pooling global para obtener representación de la secuencia completa
        # Usamos el último timestep del encoder o average pooling
        self.fc1 = nn.Linear(d_model, fc_units)
        self.relu = nn.ReLU()
        self.dropout_fc = nn.Dropout(transformer_dropout)
        
        # Capa de salida (binaria)
        self.fc2 = nn.Linear(fc_units, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass.
        
        Args:
            x: Tensor de forma (batch_size, seq_len, n_channels) o (batch_size, n_channels, seq_len)
        
        Returns:
            Tensor de forma (batch_size,) con probabilidades
        """
        batch_size = x.shape[0]
        
        # Asegurar que x tiene forma (batch_size, n_channels, seq_len) para Conv1d
        if len(x.shape) == 3 and x.shape[1] == self.seq_len and x.shape[2] == self.n_channels:
            # Forma (batch, seq_len, channels) -> transponer a (batch, channels, seq_len)
            x = x.transpose(1, 2)
        # Si ya está en forma (batch, channels, seq_len), no hacer nada
        
        # CNN1D: (batch, channels, seq_len) -> (batch, out_channels, seq_len_reduced)
        cnn_out = self.cnn(x)
        
        # Transponer para preparar para Transformer: (batch, out_channels, seq_len_reduced) 
        # -> (batch, seq_len_reduced, out_channels)
        cnn_out = cnn_out.transpose(1, 2)  # (batch, seq_len_reduced, out_channels)
        
        # Proyectar a d_model si es necesario
        cnn_out = self.cnn_projection(cnn_out)  # (batch, seq_len_reduced, d_model)
        
        # Preparar para Transformer: (batch, seq_len, d_model) -> (seq_len, batch, d_model)
        transformer_input = cnn_out.transpose(0, 1)  # (seq_len_reduced, batch, d_model)
        
        # Aplicar codificación posicional si está habilitada
        if self.pos_encoder is not None:
            transformer_input = self.pos_encoder(transformer_input)
        
        # Transformer Encoder: (seq_len, batch, d_model) -> (seq_len, batch, d_model)
        transformer_output = self.transformer_encoder(transformer_input)
        
        # Usar el último timestep o promedio global
        # Opción 1: Último timestep (más común en clasificación)
        last_timestep = transformer_output[-1]  # (batch, d_model)
        
        # Opción 2: Average pooling (comentado, usar si prefieres)
        # last_timestep = transformer_output.mean(dim=0)  # (batch, d_model)
        
        # Fully connected
        out = self.fc1(last_timestep)
        out = self.relu(out)
        out = self.dropout_fc(out)
        
        # Salida binaria
        out = self.fc2(out)
        out = self.sigmoid(out)
        
        return out.squeeze(-1)  # (batch_size,)
    
    def predict_proba(self, x: torch.Tensor) -> torch.Tensor:
        """Devuelve probabilidades (mismo que forward)."""
        return self.forward(x)
    
    def predict(self, x: torch.Tensor, threshold: float = 0.5) -> torch.Tensor:
        """Devuelve predicciones binarias."""
        proba = self.forward(x)
        return (proba > threshold).long()


In [71]:
# ========================================
# Instanciar modelo
# ========================================
def create_model(config: Dict) -> CNN1D_TransformerClassifier:
    """Crea e instancia el modelo CNN1D + Transformer."""
    model = CNN1D_TransformerClassifier(
        n_channels=config["N_CHANNELS"],
        seq_len=config["SEQ_LEN"],
        out_channels_list=config["out_channels_list"],
        kernel_sizes=config["kernel_sizes"],
        pool_sizes=config["pool_sizes"],
        use_batchnorm=config["USE_BATCHNORM"],
        cnn_activation=config["CNN_ACTIVATION"],
        d_model=config["D_MODEL"],
        nhead=config["NHEAD"],
        num_encoder_layers=config["NUM_ENCODER_LAYERS"],
        dim_feedforward=config["DIM_FEEDFORWARD"],
        transformer_dropout=config["TRANSFORMER_DROPOUT"],
        use_pos_encoding=config["USE_POS_ENCODING"],
        fc_units=config["FC_UNITS"],
    )
    
    # Contar parámetros
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"✓ Modelo creado:")
    print(f"  Parámetros totales: {total_params:,} ({total_params / 1e6:.2f}M)")
    print(f"  Parámetros entrenables: {trainable_params:,}")
    print(f"  CNN output shape: (batch, {model.cnn_output_channels}, {model.cnn_output_seq_len})")
    print(f"  Transformer input size: {model.d_model}")
    print(f"  Transformer encoder layers: {config['NUM_ENCODER_LAYERS']}")
    print(f"  Attention heads: {config['NHEAD']}")
    
    return model


---

## 5. 🏋️ Funciones de Entrenamiento y Evaluación


In [72]:
# ========================================
# Función de entrenamiento por época
# ========================================
def train_one_epoch(
    model: CNN1D_TransformerClassifier,
    train_loader: DataLoader,
    optimizer: optim.Optimizer,
    criterion: nn.Module,
    device: torch.device,
    gradient_accumulation_steps: int = 1,
    clip_grad_norm: Optional[float] = None,
    warmup_scheduler: Optional[optim.lr_scheduler.LambdaLR] = None,
    global_step: int = 0,
) -> Tuple[float, float, int]:
    """
    Entrena el modelo por una época.
    
    Returns:
        Tupla con (loss_promedio, accuracy_promedio, número_de_pasos)
    """
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    steps = 0
    
    optimizer.zero_grad()
    
    for batch_idx, (batch_x, batch_y) in enumerate(train_loader):
        batch_x = batch_x.to(device, non_blocking=True)
        batch_y = batch_y.to(device, non_blocking=True).float()
        
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # Normalizar pérdida si usas gradient accumulation
        loss = loss / gradient_accumulation_steps
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping (antes de optimizer.step)
        if clip_grad_norm is not None and clip_grad_norm > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
        
        # Actualizar pesos solo cada N pasos (gradient accumulation)
        if (batch_idx + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            
            # Actualizar warmup scheduler si existe
            if warmup_scheduler is not None:
                warmup_scheduler.step()
            
            steps += 1
        
        # Acumular métricas
        total_loss += loss.item() * batch_x.size(0) * gradient_accumulation_steps
        predictions = (outputs > 0.5).long()
        correct += (predictions == batch_y.long()).sum().item()
        total += batch_x.size(0)
    
    # Actualizar si quedan gradientes pendientes
    if (batch_idx + 1) % gradient_accumulation_steps != 0:
        optimizer.step()
        optimizer.zero_grad()
        if warmup_scheduler is not None:
            warmup_scheduler.step()
        steps += 1
    
    avg_loss = total_loss / total if total > 0 else 0.0
    avg_accuracy = correct / total if total > 0 else 0.0
    
    return avg_loss, avg_accuracy, steps


In [73]:
# ========================================
# Función de evaluación
# ========================================
def evaluate(
    model: CNN1D_TransformerClassifier,
    dataloader: DataLoader,
    criterion: nn.Module,
    device: torch.device,
) -> Tuple[float, float, np.ndarray, np.ndarray]:
    """
    Evalúa el modelo en un dataloader.
    
    Returns:
        Tupla con (loss, accuracy, y_true, y_pred)
    """
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch_x, batch_y in dataloader:
            batch_x = batch_x.to(device, non_blocking=True)
            batch_y = batch_y.to(device, non_blocking=True).float()
            
            # Forward pass
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            # Acumular métricas
            total_loss += loss.item() * batch_x.size(0)
            predictions = (outputs > 0.5).long()
            correct += (predictions == batch_y.long()).sum().item()
            total += batch_x.size(0)
            
            # Guardar predicciones y etiquetas
            all_preds.append(predictions.cpu().numpy())
            all_labels.append(batch_y.long().cpu().numpy())
    
    avg_loss = total_loss / total if total > 0 else 0.0
    avg_accuracy = correct / total if total > 0 else 0.0
    
    y_true = np.concatenate(all_labels)
    y_pred = np.concatenate(all_preds)
    
    return avg_loss, avg_accuracy, y_true, y_pred


In [74]:
# ========================================
# Función para calcular métricas completas
# ========================================
def compute_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    """
    Calcula métricas completas de clasificación.
    
    Returns:
        Diccionario con todas las métricas
    """
    accuracy = accuracy_score(y_true, y_pred)
    
    # Calcular métricas por clase usando classification_report
    report = classification_report(
        y_true, y_pred,
        target_names=["normal", "anomalo"],
        output_dict=True,
        zero_division=0
    )
    
    # Métricas para clase normal (0)
    metrics_normal = report.get("normal", {})
    precision_normal = metrics_normal.get("precision", 0.0)
    recall_normal = metrics_normal.get("recall", 0.0)
    f1_normal = metrics_normal.get("f1-score", 0.0)
    
    # Métricas para clase anómala (1)
    metrics_anom = report.get("anomalo", {})
    precision_anom = metrics_anom.get("precision", 0.0)
    recall_anom = metrics_anom.get("recall", 0.0)
    f1_anom = metrics_anom.get("f1-score", 0.0)
    
    # Métricas generales (macro avg)
    macro_avg = report.get("macro avg", {})
    precision_macro = macro_avg.get("precision", 0.0)
    recall_macro = macro_avg.get("recall", 0.0)
    f1_macro = macro_avg.get("f1-score", 0.0)
    
    # Matriz de confusión
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / max(1, tn + fp)  # TNR
    sensitivity = tp / max(1, tp + fn)   # TPR (recall de clase anómala)
    
    return {
        "accuracy": accuracy,
        "specificity": specificity,
        "sensitivity": sensitivity,
        "precision_normal": precision_normal,
        "recall_normal": recall_normal,
        "f1_normal": f1_normal,
        "precision_anom": precision_anom,
        "recall_anom": recall_anom,
        "f1_anom": f1_anom,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "confusion_matrix": cm,
    }


---

## 6. 📊 Integración con MLflow


In [75]:
# ========================================
# Configuración de MLflow
# ========================================
def setup_mlflow(config: Dict) -> str:
    """
    Configura MLflow y crea/obtiene el experimento.
    
    Returns:
        ID del experimento
    """
    # Configurar tracking URI
    if config.get("MLFLOW_TRACKING_URI") is not None:
        mlflow.set_tracking_uri(config["MLFLOW_TRACKING_URI"])
    else:
        # Usar sqlite en el directorio padre
        PARENT_DIR = Path.cwd().parent.resolve()
        TRACKING_DB = (PARENT_DIR / "mlflow.db").resolve()
        mlflow.set_tracking_uri(f"sqlite:///{TRACKING_DB.as_posix()}")
        print(f"✓ MLflow tracking URI: sqlite:///{TRACKING_DB.as_posix()}")
    
    # Crear o obtener experimento
    experiment_name = config["EXPERIMENT_NAME"]
    
    try:
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Crear directorio de artefactos
            PARENT_DIR = Path.cwd().parent.resolve()
            ARTIFACT_ROOT = (PARENT_DIR / "mlflow_artifacts").resolve()
            ARTIFACT_ROOT.mkdir(parents=True, exist_ok=True)
            experiment_id = mlflow.create_experiment(experiment_name, artifact_location=ARTIFACT_ROOT.as_uri())
            print(f"✓ Experimento MLflow creado: {experiment_name} (ID: {experiment_id})")
            print(f"  Artifact root: {ARTIFACT_ROOT.as_uri()}")
        else:
            experiment_id = experiment.experiment_id
            print(f"✓ Experimento MLflow existente: {experiment_name} (ID: {experiment_id})")
    except Exception as e:
        print(f"⚠ Error al configurar MLflow: {e}")
        experiment_id = mlflow.set_experiment(experiment_name)
    
    return experiment_id


In [76]:
# ========================================
# Función para guardar matriz de confusión como artefacto
# ========================================
def save_confusion_matrix(
    cm: np.ndarray,
    output_dir: Path,
    tag: str,
) -> Tuple[Path, Path]:
    """
    Guarda la matriz de confusión como PNG y CSV.
    
    Returns:
        Tupla con rutas (png_path, csv_path)
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Guardar como CSV
    csv_path = output_dir / f"confusion_matrix_{tag}.csv"
    df_cm = pd.DataFrame(cm, index=["Normal", "Anómalo"], columns=["Normal", "Anómalo"])
    df_cm.to_csv(csv_path)
    
    # Guardar como PNG
    png_path = output_dir / f"confusion_matrix_{tag}.png"
    fig, ax = plt.subplots(figsize=(6, 5))
    im = ax.imshow(cm, interpolation="nearest", cmap="Blues")
    ax.figure.colorbar(im, ax=ax)
    
    # Etiquetas
    ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]))
    ax.set_xticklabels(["Normal", "Anómalo"])
    ax.set_yticklabels(["Normal", "Anómalo"])
    ax.set_xlabel("Predicción")
    ax.set_ylabel("Real")
    ax.set_title(f"Matriz de Confusión - {tag.upper()}")
    
    # Añadir valores en las celdas
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j, i, f"{cm[i, j]}",
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black"
            )
    
    plt.tight_layout()
    plt.savefig(png_path, dpi=150)
    plt.close()
    
    return png_path, csv_path


In [77]:
# ========================================
# Función para guardar gráficos de curvas de entrenamiento
# ========================================
def save_training_curves(
    train_losses: List[float],
    train_accuracies: List[float],
    val_losses: List[float],
    val_f1_scores: List[float],
    output_dir: Path,
    learning_rates: Optional[List[float]] = None,
) -> Path:
    """
    Guarda gráficos de curvas de entrenamiento.
    
    Returns:
        Ruta del archivo PNG guardado
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Usar 2x3 si hay learning rates, sino 2x2
    if learning_rates:
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    else:
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    epochs = range(1, len(train_losses) + 1)
    
    # Loss
    axes[0, 0].plot(epochs, train_losses, label="Train Loss", color="blue")
    axes[0, 0].plot(epochs, val_losses, label="Val Loss", color="red")
    axes[0, 0].set_xlabel("Época")
    axes[0, 0].set_ylabel("Loss (BCE)")
    axes[0, 0].set_title("Loss de Entrenamiento y Validación")
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()
    
    # Accuracy
    axes[0, 1].plot(epochs, train_accuracies, label="Train Accuracy", color="blue")
    axes[0, 1].set_xlabel("Época")
    axes[0, 1].set_ylabel("Accuracy")
    axes[0, 1].set_title("Accuracy de Entrenamiento")
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].legend()
    
    # F1 Score en validación
    if val_f1_scores:
        axes[1, 0].plot(epochs, val_f1_scores, label="Val F1 (macro)", color="red")
        axes[1, 0].set_xlabel("Época")
        axes[1, 0].set_ylabel("F1-Score")
        axes[1, 0].set_title("F1-Score en Validación")
        axes[1, 0].grid(True, alpha=0.3)
        axes[1, 0].legend()
    
    # Learning Rate (si está disponible)
    if learning_rates:
        axes[0, 2].plot(epochs, learning_rates, label="Learning Rate", color="green")
        axes[0, 2].set_xlabel("Época")
        axes[0, 2].set_ylabel("Learning Rate")
        axes[0, 2].set_title("Learning Rate durante Entrenamiento")
        axes[0, 2].set_yscale('log')
        axes[0, 2].grid(True, alpha=0.3)
        axes[0, 2].legend()
        
        axes[1, 1].plot(epochs, train_accuracies, label="Train Accuracy", color="blue", alpha=0.7)
        axes[1, 1].set_xlabel("Época")
        axes[1, 1].set_ylabel("Accuracy")
        axes[1, 1].set_title("Train Accuracy")
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].legend()
        
        axes[1, 2].plot(epochs, train_losses, label="Train Loss", color="blue", alpha=0.7)
        axes[1, 2].plot(epochs, val_losses, label="Val Loss", color="red", alpha=0.7)
        axes[1, 2].set_xlabel("Época")
        axes[1, 2].set_ylabel("Loss")
        axes[1, 2].set_title("Train vs Val Loss")
        axes[1, 2].grid(True, alpha=0.3)
        axes[1, 2].legend()
    else:
        axes[1, 1].plot(epochs, train_accuracies, label="Train Accuracy", color="blue", alpha=0.7)
        axes[1, 1].set_xlabel("Época")
        axes[1, 1].set_ylabel("Accuracy")
        axes[1, 1].set_title("Accuracy de Entrenamiento")
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].legend()
    
    plt.tight_layout()
    
    png_path = output_dir / "training_curves.png"
    plt.savefig(png_path, dpi=150)
    plt.close()
    
    return png_path


---

## 7. 🪄 Orquestación con Prefect


In [78]:
# ========================================
# Tarea Prefect: Cargar datos
# ========================================
@task(name="load_data", log_prints=True, cache_policy=NO_CACHE)
def task_load_data(config: Dict):
    """Tarea Prefect para cargar datos desde tensors_200hz."""
    print("📂 Cargando datos...")
    # Cargar tensors desde disco
    X_train, y_train, X_val, y_val, X_test, y_test = load_tensor_data(
        config["DATA_DIR"]
    )
    
    # Crear dataloaders
    train_loader, val_loader, test_loader = create_dataloaders_from_tensors(
        X_train, y_train, X_val, y_val, X_test, y_test,
        batch_size=config["BATCH_SIZE"],
        shuffle_train=True,
    )
    
    # Convertir y_val e y_test a numpy para métricas
    y_val_np = y_val.numpy()
    y_test_np = y_test.numpy()
    
    print("✓ Datos cargados y preparados")
    return train_loader, val_loader, test_loader, y_val_np, y_test_np


In [79]:
# ========================================
# Tarea Prefect: Entrenar modelo
# ========================================
@task(name="train_model", log_prints=True, cache_policy=NO_CACHE)
def task_train_model(
    model: CNN1D_TransformerClassifier,
    train_loader: DataLoader,
    val_loader: DataLoader,
    y_val: np.ndarray,
    config: Dict,
    device: torch.device,
    experiment_id: str,
):
    """Tarea Prefect para entrenar el modelo."""
    print("🏋️ Iniciando entrenamiento...")
    print(f"  📊 Verificando DataLoaders...")
    print(f"    Train: {len(train_loader)} batches ({len(train_loader.dataset)} muestras)")
    print(f"    Val: {len(val_loader)} batches ({len(val_loader.dataset)} muestras)")
    print(f"    Device: {device}")
    
    # Mover modelo a dispositivo
    print(f"  🔄 Moviendo modelo a {device}...")
    model = model.to(device)
    print(f"  ✓ Modelo en {device}")
    
    # Optimizador y criterio
    print(f"  🔄 Inicializando optimizador y criterio...")
    optimizer = optim.Adam(
        model.parameters(),
        lr=config["LEARNING_RATE"],
        weight_decay=config["WEIGHT_DECAY"],
    )
    criterion = nn.BCELoss()
    print(f"  ✓ Optimizador y criterio listos")
    
    # Learning Rate Scheduler con Warmup
    warmup_steps = config.get("WARMUP_STEPS", 0)
    warmup_scheduler = None
    scheduler = None
    
    if warmup_steps > 0:
        # Crear función lambda para warmup linear
        def warmup_lambda(step: int) -> float:
            if step < warmup_steps:
                return float(step) / float(max(1, warmup_steps))
            return 1.0
        
        warmup_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=warmup_lambda)
        print(f"✓ Warmup Scheduler configurado: {warmup_steps} pasos")
    
    if config.get("USE_SCHEDULER", False):
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode=config.get("SCHEDULER_MODE", "max"),
            factor=config.get("SCHEDULER_FACTOR", 0.5),
            patience=config.get("SCHEDULER_PATIENCE", 5),
            min_lr=config.get("SCHEDULER_MIN_LR", 1e-6),
        )
        print(f"✓ Learning Rate Scheduler (ReduceLROnPlateau) configurado")
    
    # Gradient clipping
    clip_grad_norm = config.get("CLIP_GRAD_NORM", None)
    if clip_grad_norm is not None and clip_grad_norm > 0:
        print(f"✓ Gradient Clipping habilitado: {clip_grad_norm}")
    
    # Listas para tracking
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_f1_scores = []
    learning_rates = []
    best_f1 = 0.0
    best_model_state = None
    global_step = 0  # Contador global de pasos para warmup
    
    # Iniciar run de MLflow
    print(f"  🔄 Iniciando run de MLflow...")
    with mlflow.start_run(experiment_id=experiment_id, run_name=config["RUN_NAME"]):
        print(f"  ✓ Run de MLflow iniciado")
        # Log hiperparámetros
        print(f"  🔄 Loggeando hiperparámetros en MLflow...")
        mlflow.log_params({
            "n_channels": config["N_CHANNELS"],
            "seq_len": config["SEQ_LEN"],
            "cnn_out_channels": str(config["out_channels_list"]),
            "cnn_kernel_sizes": str(config["kernel_sizes"]),
            "cnn_pool_sizes": str(config["pool_sizes"]),
            "cnn_use_batchnorm": config["USE_BATCHNORM"],
            "cnn_activation": config["CNN_ACTIVATION"],
            "d_model": config["D_MODEL"],
            "nhead": config["NHEAD"],
            "num_encoder_layers": config["NUM_ENCODER_LAYERS"],
            "dim_feedforward": config["DIM_FEEDFORWARD"],
            "transformer_dropout": config["TRANSFORMER_DROPOUT"],
            "use_pos_encoding": config["USE_POS_ENCODING"],
            "fc_units": config["FC_UNITS"],
            "batch_size": config["BATCH_SIZE"],
            "learning_rate": config["LEARNING_RATE"],
            "num_epochs": config["NUM_EPOCHS"],
            "weight_decay": config["WEIGHT_DECAY"],
            "gradient_accumulation_steps": config.get("GRADIENT_ACCUMULATION_STEPS", 1),
            "warmup_steps": config.get("WARMUP_STEPS", 0),
            "use_scheduler": config.get("USE_SCHEDULER", False),
            "scheduler_patience": config.get("SCHEDULER_PATIENCE", 5),
            "scheduler_factor": config.get("SCHEDULER_FACTOR", 0.5),
            "clip_grad_norm": config.get("CLIP_GRAD_NORM", None),
            "cudnn_benchmark": config.get("ENABLE_CUDNN_BENCHMARK", True),
            "seed": config["SEED"],
        })
        print(f"  ✓ Hiperparámetros loggeados")
        
        # Loop de entrenamiento
        print(f"\n🚀 Iniciando loop de entrenamiento ({config['NUM_EPOCHS']} épocas)...\n")
        for epoch in range(1, config["NUM_EPOCHS"] + 1):
            print(f"\n{'='*60}")
            print(f"📅 ÉPOCA {epoch}/{config['NUM_EPOCHS']}")
            print(f"{'='*60}")
            
            # Entrenar
            print(f"  🏋️ Entrenando...")
            train_loss, train_acc, steps_taken = train_one_epoch(
                model, train_loader, optimizer, criterion, device,
                gradient_accumulation_steps=config.get("GRADIENT_ACCUMULATION_STEPS", 1),
                clip_grad_norm=clip_grad_norm,
                warmup_scheduler=warmup_scheduler,
                global_step=global_step,
            )
            global_step += steps_taken  # Actualizar contador global de pasos
            train_losses.append(train_loss)
            train_accuracies.append(train_acc)
            print(f"  ✓ Entrenamiento completado: Loss={train_loss:.4f}, Acc={train_acc:.4f} | Pasos: {global_step}")
            
            # Validar
            print(f"  📊 Validando...")
            val_loss, val_acc, y_val_true, y_val_pred = evaluate(
                model, val_loader, criterion, device
            )
            val_losses.append(val_loss)
            print(f"  ✓ Validación completada: Loss={val_loss:.4f}, Acc={val_acc:.4f}")
            
            # Calcular métricas de validación
            val_metrics = compute_metrics(y_val_true, y_val_pred)
            val_f1 = val_metrics["f1_macro"]
            val_f1_scores.append(val_f1)
            
            # Actualizar Learning Rate Scheduler
            current_lr = optimizer.param_groups[0]['lr']
            learning_rates.append(current_lr)
            
            if scheduler is not None:
                scheduler.step(val_f1)
                new_lr = optimizer.param_groups[0]['lr']
                if new_lr < current_lr:
                    print(f"  ⬇️ Learning Rate reducido: {current_lr:.6f} → {new_lr:.6f}")
            
            # Log métricas en MLflow
            mlflow.log_metrics({
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "val_loss": val_loss,
                "val_accuracy": val_metrics["accuracy"],
                "val_f1_macro": val_f1,
                "val_f1_normal": val_metrics["f1_normal"],
                "val_f1_anom": val_metrics["f1_anom"],
                "val_precision_macro": val_metrics["precision_macro"],
                "val_recall_macro": val_metrics["recall_macro"],
                "learning_rate": current_lr,
            }, step=epoch)
            
            # Guardar mejor modelo
            if val_f1 > best_f1:
                best_f1 = val_f1
                best_model_state = model.state_dict().copy()
            
            # Print progreso
            if epoch % 5 == 0 or epoch == 1:
                print(
                    f"Epoch {epoch:03d}/{config['NUM_EPOCHS']} | "
                    f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
                    f"Val Loss: {val_loss:.4f} | Val Acc: {val_metrics['accuracy']:.4f} | "
                    f"Val F1: {val_f1:.4f} | LR: {current_lr:.6f}"
                )
        
        # Cargar mejor modelo
        if best_model_state is not None:
            model.load_state_dict(best_model_state)
        
        # Guardar curvas de entrenamiento
        curves_path = save_training_curves(
            train_losses, train_accuracies, val_losses, val_f1_scores, config["OUTPUT_DIR"],
            learning_rates=learning_rates if learning_rates else None,
        )
        mlflow.log_artifact(str(curves_path))
        
        # Guardar matriz de confusión de validación
        val_metrics_final = compute_metrics(y_val_true, y_val_pred)
        cm_val_path, _ = save_confusion_matrix(
            val_metrics_final["confusion_matrix"], config["OUTPUT_DIR"], "val"
        )
        mlflow.log_artifact(str(cm_val_path))
        
        # Guardar modelo
        mlflow.pytorch.log_model(model, "model")
        
        print(f"✓ Entrenamiento completado. Mejor F1 (macro): {best_f1:.4f}")
    
    return model, train_losses, train_accuracies, val_losses, val_f1_scores, best_f1, learning_rates


In [80]:
# ========================================
# Tarea Prefect: Evaluar en test
# ========================================
@task(name="evaluate_test", log_prints=True, cache_policy=NO_CACHE)
def task_evaluate_test(
    model: CNN1D_TransformerClassifier,
    test_loader: DataLoader,
    y_test: np.ndarray,
    device: torch.device,
    config: Dict,
    experiment_id: str,
):
    """Tarea Prefect para evaluar en test."""
    print("📊 Evaluando en conjunto de test...")
    
    model = model.to(device)
    criterion = nn.BCELoss()
    
    # Evaluar
    test_loss, test_acc, y_test_true, y_test_pred = evaluate(
        model, test_loader, criterion, device
    )
    
    # Calcular métricas completas
    test_metrics = compute_metrics(y_test_true, y_test_pred)
    test_metrics["loss"] = test_loss
    test_metrics["accuracy"] = test_acc
    
    # Log en MLflow
    with mlflow.start_run(experiment_id=experiment_id, run_name=config["RUN_NAME"]):
        mlflow.log_metrics({
            "test_loss": test_loss,
            "test_accuracy": test_metrics["accuracy"],
            "test_f1_macro": test_metrics["f1_macro"],
            "test_f1_normal": test_metrics["f1_normal"],
            "test_f1_anom": test_metrics["f1_anom"],
            "test_precision_macro": test_metrics["precision_macro"],
            "test_recall_macro": test_metrics["recall_macro"],
            "test_specificity": test_metrics["specificity"],
            "test_sensitivity": test_metrics["sensitivity"],
        })
        
        # Guardar matriz de confusión de test
        cm_test_path, _ = save_confusion_matrix(
            test_metrics["confusion_matrix"], config["OUTPUT_DIR"], "test"
        )
        mlflow.log_artifact(str(cm_test_path))
    
    print("✓ Evaluación en test completada:")
    print(f"  Accuracy: {test_metrics['accuracy']:.4f}")
    print(f"  Precision (normal): {test_metrics['precision_normal']:.4f} | Recall: {test_metrics['recall_normal']:.4f} | F1: {test_metrics['f1_normal']:.4f}")
    print(f"  Precision (anómalo): {test_metrics['precision_anom']:.4f} | Recall: {test_metrics['recall_anom']:.4f} | F1: {test_metrics['f1_anom']:.4f}")
    print(f"  F1 Macro: {test_metrics['f1_macro']:.4f}")
    
    return test_metrics


In [81]:
# ========================================
# Flujo principal de Prefect
# ========================================
@flow(name="cnn_transformer_classification_training_flow", log_prints=True)
def cnn_transformer_classification_training_flow(config: Dict = None):
    """
    Flujo principal de Prefect que orquesta todo el proceso:
    1. Carga y preparación de datos
    2. Creación del modelo
    3. Entrenamiento
    4. Evaluación en test
    """
    if config is None:
        config = CONFIG
    
    print("🚀 Iniciando flujo de entrenamiento CNN1D + Transformer Clasificación...")
    print(f"Experimento MLflow: {config['EXPERIMENT_NAME']}")
    
    # Configurar MLflow
    experiment_id = setup_mlflow(config)
    
    # Cargar y preparar datos
    dataloaders = task_load_data(config)
    train_loader, val_loader, test_loader, y_val, y_test = dataloaders
    
    # Crear modelo
    print("🧠 Creando modelo...")
    model = create_model(config)
    
    # Entrenar
    model, train_losses, train_accs, val_losses, val_f1_scores, best_f1, learning_rates = task_train_model(
        model, train_loader, val_loader, y_val, config, DEVICE, experiment_id
    )
    
    # Evaluar en test
    test_metrics = task_evaluate_test(
        model, test_loader, y_test, DEVICE, config, experiment_id
    )
    
    # Guardar modelo estándar con su nombre correspondiente
    print("\n💾 Guardando modelo estándar...")
    print("="*60)
    model_path, state_dict_path, metadata_path = save_model_standard(
        model=model,
        config=config,
        output_dir=Path("./models"),
        run_name=config["RUN_NAME"],
        include_metadata=True
    )
    print("="*60)
    
    print("\n" + "="*60)
    print("✅ FLUJO COMPLETADO")
    print("="*60)
    print(f"Mejor F1 en validación: {best_f1:.4f}")
    print(f"F1 en test: {test_metrics['f1_macro']:.4f}")
    print(f"\n✓ Modelo guardado en: ./models/{config['RUN_NAME']}.pt")
    print(f"\nRevisa MLflow para ver todos los artefactos y métricas.")
    
    return {
        "model": model,
        "test_metrics": test_metrics,
        "best_f1": best_f1,
        "model_path": model_path,
        "state_dict_path": state_dict_path,
        "metadata_path": metadata_path,
    }


---

## 8. 🚀 Ejecución del Flujo Completo


In [82]:
# ========================================
# Ejecutar el flujo completo
# ========================================
# NOTA: En Jupyter Notebook, puedes ejecutar esta celda directamente
# El flujo completo incluye: carga de datos, creación del modelo, entrenamiento y evaluación

results = cnn_transformer_classification_training_flow(CONFIG)

print("\n" + "="*60)
print("✅ PROCESO FINALIZADO EXITOSAMENTE")
print("="*60)
print(f"✓ Modelo entrenado y guardado")
print(f"✓ Métricas registradas en MLflow")
print(f"✓ Artefactos guardados en: {CONFIG['OUTPUT_DIR']}")
print("\n📊 Revisa MLflow para ver todas las métricas y gráficos del entrenamiento.")


2025-11-24 19:42:51 INFO  [prefect.flow_runs] Beginning flow run 'prompt-squid' for flow 'cnn_transformer_classification_training_flow'
2025-11-24 19:42:52 INFO  [prefect.flow_runs] 🚀 Iniciando flujo de entrenamiento CNN1D + Transformer Clasificación...
2025-11-24 19:42:52 INFO  [prefect.flow_runs] Experimento MLflow: ecg_cnn_transformer_supervisado
2025-11-24 19:42:52 INFO  [prefect.flow_runs] ✓ MLflow tracking URI: sqlite:///S:/Proyecto final/mlflow.db


2025-11-24 19:42:52 INFO  [prefect.flow_runs] ✓ Experimento MLflow existente: ecg_cnn_transformer_supervisado (ID: 6)
2025-11-24 19:42:52 INFO  [prefect.task_runs] 📂 Cargando datos...
2025-11-24 19:42:52 INFO  [prefect.task_runs] ======================================================================
2025-11-24 19:42:52 INFO  [prefect.task_runs] 📂 CARGANDO DATOS DESDE tensors_200hz
2025-11-24 19:42:52 INFO  [prefect.task_runs] ======================================================================
2025-11-24 19:42:52 INFO  [prefect.task_runs] Directorio: S:\Proyecto final\data\Datos_supervisados\tensors_200hz
2025-11-24 19:42:52 INFO  [prefect.task_runs] 
⏳ Cargando tensors desde disco...
2025-11-24 19:42:58 INFO  [prefect.task_runs] 
✓ Datos cargados:
2025-11-24 19:42:58 INFO  [prefect.task_runs]   X_train: torch.Size([270668, 2000, 3]) | y_train: torch.Size([270668]) (normales: 135334, anómalos: 135334)
2025-11-24 19:42:58 INFO  [prefect.task_runs]   X_val:   torch.Size([58001, 2000, 3


✅ PROCESO FINALIZADO EXITOSAMENTE
✓ Modelo entrenado y guardado
✓ Métricas registradas en MLflow
✓ Artefactos guardados en: outputs

📊 Revisa MLflow para ver todas las métricas y gráficos del entrenamiento.


---

## 9. 💾 Guardado de Modelo

Después de entrenar, puedes guardar el modelo en diferentes formatos:
1. **Modelo estándar** - Para uso local o despliegue personalizado
2. **Modelo para AWS SageMaker** - Formato listo para desplegar en SageMaker


In [83]:
# ========================================
# Función para guardar modelo estándar
# ========================================
def save_model_standard(
    model: CNN1D_TransformerClassifier,
    config: Dict,
    output_dir: Path,
    run_name: str = None,
    include_metadata: bool = True,
):
    """
    Guarda el modelo completo y metadatos en formato estándar.
    
    Args:
        model: Modelo entrenado
        config: Diccionario de configuración
        output_dir: Directorio donde guardar
        run_name: Nombre del run (para el archivo)
        include_metadata: Si True, guarda también config.json
        
    Returns:
        Tuple con (model_path, state_dict_path, metadata_path)
    """
    from datetime import datetime
    
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    if run_name is None:
        run_name = config.get("RUN_NAME", "cnn_transformer_model")
    
    # 1. Guardar modelo completo (.pt) con state_dict y config
    model_path = output_dir / f"{run_name}.pt"
    torch.save({
        'model_state_dict': model.state_dict(),
        'model_config': {
            'n_channels': config['N_CHANNELS'],
            'seq_len': config['SEQ_LEN'],
            'out_channels_list': config['out_channels_list'],
            'kernel_sizes': config['kernel_sizes'],
            'pool_sizes': config['pool_sizes'],
            'use_batchnorm': config['USE_BATCHNORM'],
            'cnn_activation': config['CNN_ACTIVATION'],
            'd_model': config['D_MODEL'],
            'nhead': config['NHEAD'],
            'num_encoder_layers': config['NUM_ENCODER_LAYERS'],
            'dim_feedforward': config['DIM_FEEDFORWARD'],
            'transformer_dropout': config['TRANSFORMER_DROPOUT'],
            'use_pos_encoding': config['USE_POS_ENCODING'],
            'fc_units': config['FC_UNITS'],
        },
        'training_config': {
            'batch_size': config['BATCH_SIZE'],
            'learning_rate': config['LEARNING_RATE'],
            'num_epochs': config['NUM_EPOCHS'],
            'weight_decay': config['WEIGHT_DECAY'],
        }
    }, model_path)
    print(f"✓ Modelo completo guardado: {model_path}")
    
    # 2. Guardar solo el state dict (más liviano)
    state_dict_path = output_dir / f"{run_name}_state_dict.pt"
    torch.save(model.state_dict(), state_dict_path)
    print(f"✓ State dict guardado: {state_dict_path}")
    
    # 3. Guardar metadatos como JSON (opcional)
    metadata_path = None
    if include_metadata:
        metadata = {
            'model_type': 'CNN1D_Transformer_Classifier',
            'run_name': run_name,
            'experiment_name': config.get('EXPERIMENT_NAME', ''),
            'date': str(datetime.now()),
            'model_config': {
                'n_channels': config['N_CHANNELS'],
                'seq_len': config['SEQ_LEN'],
                'out_channels_list': config['out_channels_list'],
                'kernel_sizes': config['kernel_sizes'],
                'pool_sizes': config['pool_sizes'],
                'use_batchnorm': config['USE_BATCHNORM'],
                'cnn_activation': config['CNN_ACTIVATION'],
                'd_model': config['D_MODEL'],
                'nhead': config['NHEAD'],
                'num_encoder_layers': config['NUM_ENCODER_LAYERS'],
                'dim_feedforward': config['DIM_FEEDFORWARD'],
                'transformer_dropout': config['TRANSFORMER_DROPOUT'],
                'use_pos_encoding': config['USE_POS_ENCODING'],
                'fc_units': config['FC_UNITS'],
            },
            'training_config': {
                'batch_size': config['BATCH_SIZE'],
                'learning_rate': config['LEARNING_RATE'],
                'num_epochs': config['NUM_EPOCHS'],
            }
        }
        metadata_path = output_dir / f"{run_name}_metadata.json"
        with open(metadata_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, ensure_ascii=False)
        print(f"✓ Metadatos guardados: {metadata_path}")
    
    return model_path, state_dict_path, metadata_path


---

## ✅ Resumen del Notebook

Este notebook implementa un **CNN1D + Transformer Encoder** para clasificación binaria supervisada de señales ECG.

**Características principales:**
- ✅ Arquitectura híbrida: CNN1D para características locales + Transformer Encoder para dependencias temporales
- ✅ Codificación posicional senoidal
- ✅ Entrenamiento supervisado con etiquetas (0=normal, 1=anómalo)
- ✅ Integración completa con MLflow para tracking de experimentos
- ✅ Orquestación con Prefect 2.x
- ✅ Soporte para GPU (CUDA)

**Instrucciones de uso:**
1. Ejecuta la celda de **Setup CUDA** primero
2. Configura los parámetros en la sección de **CONFIGURACIÓN GENERAL**
3. Ajusta la ruta `DATA_DIR` a tu carpeta de datos (`Datos_supervisados/tensors_200hz`)
4. Ejecuta todas las celdas en orden
5. El modelo entrenado se guardará automáticamente y se registrará en MLflow
